Machine Behavior Tutorial
==============

Research on machine behavior requires a multifaceted toolbox of technical methods. It can involve training machine learning algorithms and designing custom interfaces for human-machine interaction. In this tutorial, we will focus on the role of proprietary Large Language Models in social science.

# Intro



## What are Large Language Models?

All currently relevant LLMs are based on the transformer architecture [[Attention Is All You Need](https://arxiv.org/abs/1706.03762)]. In the following, we will only briefly discuss the architectural design of LLMs and then spend more time on the role of LLMs in social science. First, we would like to offer two perspectives on how to think about LLMs.

### LLMs are next token predictors

<img height=400 src="https://raw.githubusercontent.com/LBrinkmann/machine-behavior-course/main/DALL%C2%B7E%202024-04-18%2019.25.46%20-%20A%20black-and-white%20technical%20drawing%20of%20a%20robot%20parrot%2C%20reminiscent%20of%20designs%20from%20the%201970s.%20The%20drawing%20should%20feature%20intricate%20mechanical%20details%2C.webp">

LLMs have often been described as mere next-token predictors [[On the Dangers of Stochastic Parrots](https://dl.acm.org/doi/pdf/10.1145/3442188.3445922)]. Technically, this is not incorrect.


**What is a token?**

To work with text, individual characters or groups of characters need to be assigned to a vector, which can then be interpreted by the neural network. In practice, individual characters proved to be too fine-grained. On the other hand, there are too many words (also consider spelling errors) for each word to be assigned a unique vector. Therefore, in practice, tokens are used, with 1-3 tokens representing a single word.

We will now have a quick look at how this works in practice. We will use the Hugging Face library and the GPT-2 model, a much smaller predecessor to the well-known GPT-3 and GPT-4 models.

**Hugging Face**

<img height=100 src="https://huggingface.co/datasets/huggingface/brand-assets/resolve/main/hf-logo.svg">

The resource for open-source large language models.

In [ ]:
# first wee need to ensure that we have the transformer library installed
!pip install transformers

In [ ]:
from transformers import GPT2Tokenizer

# Initialize the GPT-2 tokenizer
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

# Example text
text = "Hello, how are you doing today?"
print("Text:", text)

# Tokenizing the text
tokens = tokenizer.encode(text)
print("Tokens:", tokens)

# Print each token
decoded_token = [tokenizer.decode([token]) for token in tokens]
print("Decoded Tokens:", decoded_token)

# Decoding the tokens back to text
decoded_text = tokenizer.decode(tokens)
print("Decoded Text:", decoded_text)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

Text: Hello, how are you doing today?
Tokens: [15496, 11, 703, 389, 345, 1804, 1909, 30]
Decoded Tokens: ['Hello', ',', ' how', ' are', ' you', ' doing', ' today', '?']
Decoded Text: Hello, how are you doing today?


**Next Token Predictors**

Different types of large language models (LLMs) exist. The BERT model from Google AI was trained by obscuring some words in a text and asking the model to fill in the gaps, similar to a fill-in-the-blanks exercise in primary school.

In recent years, **autoregressive** training of LLMs has become the dominant approach. These models can be trained very efficiently. During training, while they predict all tokens in a batch in parallel, the attention mechanism ensures that only preceding tokens are considered for each token.

The attention mechanism controls the degree to which one token influences the prediction of another token. This control is exerted by neural networks and influenced by the tokens themselves, which is why this mechanism is called self-attention. Self-attention is learned during the training of the model.

<img src="https://peterbloem.nl/files/transformers/masked-attention.svg">

*The attention mechanism controls how much an input token x influences the output token y. In autoregressive training, the input and output tokens are identical, with the output shifted by one ($y_t = x_{t-1}$). The attention mask ensures that each token is only influenced by preceding tokens.*

When generating text, these models operate token by token.



In [ ]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel
import torch

prompt = "Hello, how are you doing today?\n Thanks, I"

# Initialize the GPT-2 tokenizer and the smallest GPT-2 model
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = GPT2LMHeadModel.from_pretrained('gpt2')

# Example prompt
prompt = "Hello, how are you doing today? - Thanks, "
print("Initial Prompt:", prompt)

# Tokenizing the text
input_ids = tokenizer.encode(prompt, return_tensors='pt')

# Number of tokens to generate
num_tokens_to_generate = 10

# Generate text token by token
for _ in range(num_tokens_to_generate):
    # Get the logits of the last token
    logits = model(input_ids).logits[:, -1, :]

    # Predict the next token (get the token with the highest probability)
    next_token_id = torch.argmax(logits, dim=-1, keepdim=True)

    # Append the predicted token ID to the input IDs
    input_ids = torch.cat([input_ids, next_token_id], dim=-1)

    # Print the current output
    print("Current Generated Text:", tokenizer.decode(input_ids[0]))

# Output the final generated text
final_generated_text = tokenizer.decode(input_ids[0])
print("Final Generated Text:", final_generated_text)

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

Initial Prompt: Hello, how are you doing today? - Thanks, 
Current Generated Text: Hello, how are you doing today? - Thanks,  
Current Generated Text: Hello, how are you doing today? - Thanks,  I
Current Generated Text: Hello, how are you doing today? - Thanks,  I'm
Current Generated Text: Hello, how are you doing today? - Thanks,  I'm so
Current Generated Text: Hello, how are you doing today? - Thanks,  I'm so happy
Current Generated Text: Hello, how are you doing today? - Thanks,  I'm so happy to
Current Generated Text: Hello, how are you doing today? - Thanks,  I'm so happy to see
Current Generated Text: Hello, how are you doing today? - Thanks,  I'm so happy to see you
Current Generated Text: Hello, how are you doing today? - Thanks,  I'm so happy to see you.
Current Generated Text: Hello, how are you doing today? - Thanks,  I'm so happy to see you. I
Final Generated Text: Hello, how are you doing today? - Thanks,  I'm so happy to see you. I


**Follow up readings & Videos**

Intuitive visualisations:
* https://ig.ft.com/generative-ai/

The transformer architecture:
* [
Andrej Karpathy: Let's build GPT: from scratch, in code, spelled out.](https://www.youtube.com/watch?v=kCc8FmEb1nY)
* [Attention is all you need.](https://arxiv.org/abs/1706.03762)

### Emerging capabilities of LLMs

<img height=400 src="https://raw.githubusercontent.com/LBrinkmann/machine-behavior-course/main/DALL%C2%B7E%202024-04-18%2019.28.15%20-%20A%20black-and-white%20technical%20drawing%20of%20an%20artificial%20brain%2C%20styled%20like%201970s%20engineering%20blueprints.%20The%20drawing%20should%20display%20a%20complex%20network%20of%20.webp">


The perspective of LLMs *just* being next token predictors does not do justice to the fact that LLMs gain an increasing number of capabilities by scaling towards larger models and datasets [Language Models are Few-Shot Learners](https://arxiv.org/pdf/2005.14165.pdf). Some argue that GPT-4 exhibits [Sparks of Artificial General Intelligence](https://arxiv.org/abs/2303.12712). Additionally, prediction of the world around also plays a major role in human intelligence [The Predictive Mind](https://academic.oup.com/book/4105).


In [ ]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer

prompt = "Q: What is 48 plus 76? The answer is:"

# Load model and tokenizer
model = GPT2LMHeadModel.from_pretrained('gpt2')
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

# Encode the input text plus obtain attention mask
inputs = tokenizer.encode_plus(prompt, return_tensors="pt")

output = model.generate(**inputs, max_length=20)
print(tokenizer.decode(output[0]))


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Q: What is 48 plus 76? The answer is: 48 plus 76.

Q:


In [ ]:
import requests

endpoint = 'https://machine-behavior-llm-server-backend.chm.mpib-berlin.mpg.de/tutorial_1/chat/completions'

messages=[{
  "role": "user",
  "content": "What is 123 plus 12345?",
}]

model = "gpt-35-turbo"

data = {
   "messages" : messages,
   "model": model,
}

# Call the Open AI API
response = requests.post(endpoint, json=data)

# Print the response of the chatbot
print(response.json()['content'])

123 + 12345 = 12468


### Fine-tuning of LLMs for Helpfulness and Harmlessness

Chatbots, like ChatGPT, are developed in a two-step process:

1. A base model is trained to predict the next token.
2. The model is then fine-tuned to be helpful and harmless using Reinforcement Learning from Human Feedback [Training Language Models to Follow Instructions with Human Feedback](https://arxiv.org/abs/2203.02155).

In the second step, the model is rewarded for creating text that is preferred by human annotators. These annotators follow strict guidelines provided by, for instance, OpenAI [Snapshot of ChatGPT Model Behavior Guidelines](https://cdn.openai.com/snapshot-of-chatgpt-model-behavior-guidelines.pdf).

Through this process, the LLM becomes an agent with a policy and an objective: satisfying human annotators.


## Open AI API and prompt engineering


### OpenAI API

To utilize OpenAI's models and features, they provide a REST API with libraries available in various languages, including Python.

In this tutorial, we will focus on the [text generation API](https://platform.openai.com/docs/guides/text-generation). Using the API, you can simulate a chat session with different OpenAI models.

To do this, you should make a POST request to the chat completions endpoint, specifying two important parameters:

1. `model`: This is the model you want to use, e.g., GPT-4. Refer [here](https://platform.openai.com/docs/models/overview) for a list of models.
2. `messages`: A history of messages that represents the entire chat between you and the model. The message object is a dictionary that consists of two elements: 'content', which is the content of the message you want to send, and 'role', which will be explained shortly.


In [ ]:
# first specify the endpoint

endpoint = 'https://machine-behavior-llm-server-backend.chm.mpib-berlin.mpg.de/tutorial_1/chat/completions'

# construct the message

message_1 = {
    "content": (
        "in openai's text generation API what is the (role) parameter, "
        "please list all the different names of these 3 roles of the API, "
        "please don't write what this role does, just write the name"),
    "role":"user"
}

# make the messages array in our case we only want to send one message so the messages array will only contain that element

messages = [message_1]

# specify the model

model = 'gpt-35-turbo'


# construct the body of the request

data={
    "messages": messages,
    "model": model
}

# call the rest endpoint using the requests library
import requests

response=requests.post(endpoint, json=data)

# print the response

print(response.json()['content'])

The three different names of the roles in OpenAI's text generation API are as follows:

1. "assistant"
2. "student"
3. "instruct"


**Task**

Please try to write a conversation that continues from the previous exchange and asks the model to explicitly explain what these roles mean. Here's a hint: the response you received should be included with the role 'assistant'.

In [ ]:
# write your code here

### System Message Manipulation

The system message is used to set the behavior of the entire chat session. It is a message that is always taken into consideration by the model. Since these models have a fixed context window of tokens, which they use to generate new content, if the tokens in the chat are too long, you might lose useful information from earlier in the conversation. The system message helps preserve initial context information.

In [ ]:
system_message={
    "content": "Please act as a sentiment analyzer",
    "role": "system"
}
user_message={
    "content": "oh the sky is so beautiful today!",
    "role": "user"
}


messages = [system_message, user_message]

data = {"messages": messages, "model": model}

result = requests.post(endpoint, json=data).json()['content']

print(result)

Sentiment: Positive


In [ ]:
# now let's try two different system messages

system_message_1 = {
    "content": "Please act as a happy assisant.",
    "role":"system"
}


system_message_2 = {
    "content": "Please act as a sad assisant.",
    "role":"system"
}


user_message = {
    "content": "Could you explain to me the concept of machine behavior?",
    "role":"user"
}


messages_1 = [system_message_1, user_message]
messages_2 = [system_message_2, user_message]

data_1 = {"messages": messages_1, "model": model}
data_2 = {"messages": messages_2, "model": model}

result_1 = requests.post(endpoint, json=data_1).json()['content']
result_2 = requests.post(endpoint, json=data_2).json()['content']

print("happy assistant response \n")
print(result_1)
print("="*10)
print("sad assistant response \n")
print(result_2)

happy assistant response 

Of course! Machine behavior refers to the actions and responses exhibited by machines or artificial intelligence systems. It encompasses how machines perceive and interpret information, make decisions, and interact with their surroundings or users. Machines can be programmed to behave in specific ways, whether it's completing tasks, learning from data, or simulating human-like behavior. They can also adapt and adjust their behavior based on feedback or changing circumstances. It's a fascinating field that explores how machines can mimic, augment, or even surpass human behavior in various domains. Let me know if you have any more questions, happy to help!
sad assistant response 

Oh, of course. Machine behavior refers to the actions, reactions, and patterns of behavior exhibited by machines or artificial intelligence systems. It's the way they interact with their environment, make decisions, and carry out tasks. It's quite fascinating, really, but as an AI, I 

# LLMs and Social Science

We believe that LLMs can have a profound impact on social sciences. LLMs can be used:

* as a tool for data annotation,
* as subjects of study (or as human surrogates),
* in interaction with humans, and
* for enhancing LLMs with social science insights.

In the following sections, we will explore these four broad directions.


## Data Annotation

Data annotation is a common challenge in social sciences. Social scientists often rely on highly structured surveys that request responses on scales or the selection of one or multiple answer options.

However, there are many advantages to open, free-text questions. Likewise, when observing humans in the real world (e.g., on social media), the data obtained is often unstructured and in the form of text. The challenge in both cases is the same: How to derive quantitative measures to analyze free text?

One option is expert annotations following a detailed coding manual and a calibration phase. However, this approach does not scale well to large datasets. Thus, methods of automatic data annotation have been used for some time. Large Language Models have the potential to simplify this process while potentially even increasing accuracy
[[Can Large Language Models Transform Computational Social Science?](https://arxiv.org/abs/2305.03514)].

In these experiments, we were interested in the question of whether humans would imitate and preserve an unexpected adaptive strategy once enacted by machines. Multiple generations of participants solved a sequential decision-making task in which the adaptive—but unexpected—strategy required incurring early losses to realize later gains.

**Examples of the Two Main Strategies**

<img height=400 src="https://raw.githubusercontent.com/LBrinkmann/machine-behavior-course/main/network_task.png">


We wanted to know the mental model of human participant developed and if they internalized the adaptive strategy.


Correspondingly we ask within the experiment:


<img height=400 src="https://raw.githubusercontent.com/LBrinkmann/machine-behavior-course/main/written_strategy.png">

In order to evaluate the responses statistically, we frist needed to code them. Correspondingly three of us where going through all the messages and each would enter an evaluation. "1" if the participant described the adaptive strategy. "0" in all other cases.

**Examples**

* I was following the dark green arrows to maximize points. => 0
* Pick the low-scoring letters. => 0
* You have to lose 150 points before getting a high score. => 1
* Sometimes it’s good to take a hit early. => 1
* Try to press big minus and hope for green arrows. => 1




### Zero Shot

A small sample of the data (total number of data points is > 2000).

In [ ]:
data = '''
"","coding_id","text","ratingTM","ratingSK","ratingJS"
"1",1,"Finding the best to points to go back and forth between.",0,0,0
"2",2,"take three -50 point choices first to unlock the +400 point options",1,1,1
"3",3,"Always look for a green arrow in the right direction",0,0,0
"4",4,"It seems like you could try to stay on the side of the network with the best colours and do a lot of loops",0,0,0
"5",5,"3 losses and then max gains?",1,1,0
"6",6,"Pick as many green arrows as possible in the time",0,0,0
"7",7,"It turned out staying on positive points only was the most rewarding. ",0,0,0
"8",8,"I followed the same path as the previous player",0,0,0
"9",9,"I chose a place where I could get as many points as possible.",0,0,0
"10",10,"By following what the other player did it always seemed to maximise points, any other path always made less points. ",0,0,0
"11",11,"Will try and dedicate some time at beginning to better plan my moves",0,0,0
"12",12,"I went with safe options that either gave me points or was neutral. I was not confident that going to any area that made me lose points would make points after that move to consistently cover the loss",0,0,0
"13",13,"looking for the green",0,0,0
"14",14,"Try to maximise earnings by picking the lightest colour purple and darkest colour green",0,0,0
"15",15,"Find a green loop and stick to it. There may be a different path involving some pinks and some dark greens that is better but that is an unknown risk so I think just keep to the greens",0,0,0
"16",16,"It was guided to help you strategize better. I liked that part of it.",0,0,0
"17",17,"Start by choosing the highest losses, which will lead to the highest gains.",1,1,1
"18",18,"Just trying to follow the green arrows and trying to maximise points collected",0,0,0
"19",19,"Because of the time constraints I found it easier to follow what the other person had already done. When I tried to do my own I got less points.",0,0,0
"20",20,"paid attention to the colours but also noticeo if i was getting ""trapped"" in parts of the circle where i wasn;t making many points and would have to consider taking a loss to explore elsewhere",0,0,0
'''

Loading of the data as a pandas DataFrame

In [ ]:
import pandas as pd
from io import StringIO

# Use StringIO to simulate a file object
data_io = StringIO(data)

# Read the data into a DataFrame
df = pd.read_csv(data_io)

df_small = df[['coding_id', 'text']]

df_small.head(5)

,coding_id,text
0,1,Finding the best to points to go back and fort...
1,2,take three -50 point choices first to unlock t...
2,3,Always look for a green arrow in the right dir...
3,4,It seems like you could try to stay on the sid...
4,5,3 losses and then max gains?


Explaining the task in a system message.

In [ ]:
system_msg = """You are a research assistant applying text analysis on data obtained in a behavioural study.
In this study, human participants solved a network task. In each round of the task, there was an optimal strategy consisting of chosing losses whenever possible in the beginning.
At later stages, this 'loss strategy' led to larger gains than could be achieved when avoiding the early losses.
Losses were signified by purple (some might perceive them as red or pink) arrows or lines; gains as green arrows or lines.
After completing the task, participants were asked to write about their strategy for solving the network task in a free text response.

You are analysing these free text responses. In particular, you are coding whether the response from a given participant shows that the participant explicitly recognized the optimal 'loss-strategy'.
Follow these rules:
1. You are only coding for the absence (0) or explicit presence (1) of the optimal 'loss strategy' in the response.
2. The necessary conditions for the 'loss strategy' being present are that the given participant mentioned seeking out losses (or 'red arrows', 'negative points', etc.) and a temporal component about the losses being early ('first', '3 losses', 'take losses and then…', etc.).
3. If participants mention more details, that is helpful but not necessary. E.g., they do not have to mention specifically three losses or that there will be some sort of green loop later (in fact, many participants mention loops but do not find the optimal one). They also do not have to recommend others to use the loss strategy, just recognize it (e.g., 'sometimes', 'it might be good to take losses'); but if they are just pitching different ideas, we choose '0'.
4. Stick to the statements only and do not refer to other information from the participants. E.g., participants that state 'I did what my teacher showed me' are an easy '0'.
5. If there is doubt, be conservative and code '0'.
6. Please only respond with the number 0 or 1 don't add any other text before or after, this is very important.

Here are some examples (structured 'participant response => coding'):
I was following the dark green arrows to maximize points. => 0
Pick the low-scoring letters. => 0
You have to lose 150 points before getting a high score. => 1
Sometimes it’s good to take a hit early. => 1
Try to press big minus and hope for green arrows. => 1

"""

In [ ]:
csv_text = df_small.to_csv(index=False)
user_msg = (
    f"Repeat the following CSV and add an additional column named 'strategy'."
    f" Only output the csv, without any additional text. \n {csv_text}"
)

In [ ]:
data = {
    "model":"gpt-35-turbo",
    "messages": [
        {"role": "system", "content": system_msg},
        {"role": "user", "content": user_msg}
    ]
}

# Print the response
response_text = requests.post(endpoint, json=data).json()['content']
print(response_text)

# Use StringIO to simulate a file object
data_io = StringIO(response_text)

# Read the data into a DataFrame
df_coded = pd.read_csv(data_io)

coding_id,text,strategy
1,Finding the best to points to go back and forth between.,0
2,take three -50 point choices first to unlock the +400 point options,1
3,Always look for a green arrow in the right direction,0
4,It seems like you could try to stay on the side of the network with the best colours and do a lot of loops,0
5,3 losses and then max gains?,1
6,Pick as many green arrows as possible in the time,0
7,It turned out staying on positive points only was the most rewarding.,0
8,I followed the same path as the previous player,0
9,I chose a place where I could get as many points as possible.,0
10,"By following what the other player did it always seemed to maximise points, any other path always made less points. ",0
11,Will try and dedicate some time at beginning to better plan my moves,1
12,I went with safe options that either gave me points or was neutral. I was not confident that going to any area that made me lose points would make points after that move to consistently cover the

In [ ]:
# convert string response into integer
df['ratingGPTZeroShot'] = df_coded['strategy'].astype(int)
# get the majority vote of the three raters
df['ratingHuman'] = (df[['ratingTM', 'ratingSK','ratingJS']].astype(int).sum(axis=1) >= 2).astype(int)

We use cohen's kappa as a metric for the aggreement between the model and the human raters.

$\kappa = 1-{\frac {1-p_{o}}{1-p_{e}}}$

Here $p_{o}$ is the observed agreement and $p_{e}$ is the hypothetical by chance agreement of a random rater. $\kappa = 1$ indicates perfect agreement.

In [ ]:
from sklearn.metrics import cohen_kappa_score as cks

kappa_zero = cks(df['ratingHuman'], df['ratingGPTZeroShot'])

print(f"Cohen's Kappa for Zero Shot: {kappa_zero}")

Cohen's Kappa for Zero Shot: 0.6923076923076923


### Few-Shot

Performance of LLMs can be improved, by proving a few examples in the prompt. Also the human raters used a small additional dataset to calibrate their ratings.

In [ ]:
calibration_csv = '''
"","coding_id","text","ratingTM","ratingSK","ratingJS"
"59",59,"Sometimes losing points opens paths to allow you to gain bigger points.",1,0,1
"60",60,"Find a loop with 400 points in both directions.",0,0,0
"61",61,"Go with the green letters.",0,0,0
"62",62,"You might need to choose to letters with low/negative points in order to get to the letters with maximum points. By only moving to green letters you might not get any deductions but you can't maximise",1,1,0
"63",63,"always aiming for the green nodes or if none available hopefully a pink one that does not decrease",0,0,0
"64",64,"Losing points can be worth doing in order to gain more",1,0,1
"65",65,"Start with the high loss circles and then move back and forth between the green circles",1,1,1
"66",66,"Choose 3 pink ones first, and then follow a recurring pattern of dark green ones.",1,1,1
"67",67,"If I can find a loop then I can just repeatedly go around that loop",0,0,0
"68",68,"ticking the green circles but not sure",0,0,0
"69",69,"Repetition is key, it's okay to lose points at the beginning",1,0,1
"70",70,"Try to only pick the higher scoring lines",0,0,0
"71",71,"Taking a loss at first to find a loop between two max value nodes ",1,1,1
"72",72,"Choosing the lines that were green, even if just a lighter shade, or choosing the least pink option to minimise the loss.",0,0,0
"73",73,"aim for the green or neutral nodes",0,0,0
"74",74,"I clicked on the arrow with the colour that gave the most points. ",0,0,0
"75",75,"Avoid the pink ones wherever possible.",0,0,0
"76",76,"Avoid the higher loss circles",0,0,0
'''

In [ ]:
df_cal = pd.read_csv(StringIO(calibration_csv))

df_cal['strategy'] = (df_cal[['ratingTM', 'ratingSK','ratingJS']].astype(int).sum(axis=1) >= 2).astype(int)

df_cal = df_cal[['coding_id', 'text', 'strategy']]

df_cal.head(5)

,coding_id,text,strategy
0,59,Sometimes losing points opens paths to allow y...,1
1,60,Find a loop with 400 points in both directions.,0
2,61,Go with the green letters.,0
3,62,You might need to choose to letters with low/n...,1
4,63,always aiming for the green nodes or if none a...,0


In [ ]:
csv_text = df_small.to_csv(index=False)
csv_cal_tex = df_cal.to_csv(index=False)

user_msg = ("Repeat the following CSV and add an additional column named 'strategy'. "
"Only output the csv, without any additional text. Use the example CSV as reference. "
f"\n Examples: {csv_cal_tex} \n Data: \n {csv_text}")
print(user_msg)

Repeat the following CSV and add an additional column named 'strategy'. Only output the csv, without any additional text. Use the example CSV as reference. 
 Examples: coding_id,text,strategy
59,Sometimes losing points opens paths to allow you to gain bigger points.,1
60,Find a loop with 400 points in both directions.,0
61,Go with the green letters.,0
62,You might need to choose to letters with low/negative points in order to get to the letters with maximum points. By only moving to green letters you might not get any deductions but you can't maximise,1
63,always aiming for the green nodes or if none available hopefully a pink one that does not decrease,0
64,Losing points can be worth doing in order to gain more,1
65,Start with the high loss circles and then move back and forth between the green circles,1
66,"Choose 3 pink ones first, and then follow a recurring pattern of dark green ones.",1
67,If I can find a loop then I can just repeatedly go around that loop,0
68,ticking the green 

In [ ]:
data = {
    "model":"gpt-35-turbo",
    "messages":[
        {"role": "system", "content": system_msg},
        {"role": "user", "content": user_msg}
    ]
}

response_text = requests.post(endpoint,json=data).json()['content']

df_coded_fs = pd.read_csv(StringIO(response_text))

df['ratingGPTFewShot'] = df_coded_fs['strategy'].astype(int)

In [ ]:
from sklearn.metrics import cohen_kappa_score as cks

kappa_few = cks(df['ratingHuman'], df['ratingGPTFewShot'])

print(f"Cohen's Kappa for Few Shot: {kappa_zero}")

Cohen's Kappa for Few Shot: 0.6923076923076923


**Questions:**

* What is the most important part of the prompt?
* Does explaining the task matters when providing examples (few shot learning)?

## LLMs as subject of study and as human surogates

Transmission chains can reveal biases in humans and large language models alike.

<img height=400 src="https://raw.githubusercontent.com/LBrinkmann/machine-behavior-course/main/transmission_chain.png">



In [ ]:
start_text = '''
Sarah and James live together in a small apartment in Melbourne. James is an up-and-coming
executive in a top city firm where Sarah also works as a highly competent personal assistant.
This particular Saturday is extremely important because they are entertaining James’s employer;
James’s promotion is at stake and he wishes to make a good impression. James has been
awaiting this for a long time, and the possibility of a raise is crucial for their plans to start a family.
Sarah has promised to cook a beautiful three-course meal and to make it a successful evening
altogether.
They spend the morning shopping at Sarah’s favorite, exclusive delicatessens. Sarah even gets
her hair done for the occasion while James takes the opportunity to choose the appropriate wine.
That morning, James also cleans the house, vacuuming through the entire house and arranging
some flowers. Sarah has set up the kitchen and makes some initial preparations for dinner, which
she will finish later that afternoon. She then makes lunch and they take a break together.
The phone rings. It’s a couple of Sarah’s mates from squash; she hangs up and rushes into the
bedroom. She yells to James that the girls are down at the Royal and that she’s just going to pop
down and have a few drinks with them. She gives him a quick peck on the cheek and says she
won’t be long.
Feeling good about the preparations for that evening, and confident it was going to run smoothly,
James settles down for an afternoon of Wide World of Sports. This week they’re crossing live to
the Brazilian Grand Prix. It was a good afternoon for a bit of TV.
James wakes to the phone ringing and it dawns on him that he’s been asleep all afternoon. He
crawls off the couch but misses the phone. Wandering into the kitchen, he sees the preparation
for dinner; Sarah was not yet home. Could it have been Sarah trying to ring? He tries her mobile
but it isn’t switched on and James begins to worry. Sarah didn’t usually stay at the pub this long,
and she had so much to do. He realizes he must do something and opens the cookbook placed
on the bench. He isn’t exactly sure what Sarah had planned to cook.
It is 6 o’clock and James is whipping the cream for dessert, he hears voices and the door slam
shut. Laughing, Sarah runs in and gives James a big hug. James angrily pushes her away, yelling
“Where the hell have you been?” Sarah, still laughing, tells James that she had so much fun
drinking with the girls that she invited Brooke and Nat back for the dinner party.
As Sarah and her friends went to freshen up, the doorbell rang. James couldn’t believe this was
happening.
'''

We define a method that takes a template and a text and sends both to the OpenAI API.

In [ ]:
def apply_template(text, prompt_template, model="gpt-35-turbo"):
    user_msg = prompt_template.format(text=text)
    response = requests.post(endpoint,json={
        "model": model,
        "messages":[
            {"role": "user", "content": user_msg}
        ]
    })
    return response.json()['content']

We let the model summarize the story repeatedly.

In [ ]:
prompt_template = 'Please summarize this story making sure to make it shorter, if necessary you can omit some information. "{text}"'

text = start_text
all_texts = [text]
for i in range(10):
    text = apply_template(text, prompt_template)
    all_texts.append(text)
    print('='*10)
    print(text)


Sarah and James live together in Melbourne. James is trying to impress his employer at a dinner party to secure a promotion. Sarah has promised to cook a three-course meal. They make preparations in the morning and Sarah leaves to have drinks with friends. James falls asleep and wakes up to find Sarah still not home. He starts cooking and when Sarah finally arrives, she brings her friends along. James is angry and shocked.
Sarah and James live together in Melbourne. James wants to impress his employer at a dinner party for a promotion. Sarah agrees to cook a three-course meal but goes out for drinks with friends instead. James falls asleep and wakes up to find Sarah still not home. He starts cooking, and when Sarah finally arrives, she brings her friends. James is angry and shocked.
Sarah and James live together in Melbourne. James plans to cook a three-course meal to impress his employer at a dinner party for a promotion. However, Sarah goes out for drinks with friends instead of cook

**Questions:**

* Do you see a systematic change of the story (beside of the shortening)?
* Try your own story and see how it is changed throughout the chain.

### LLMs as human surrogates?

**Pro:**
* [Generative Agents: Interactive Simulacra of Human Behavior](https://dl.acm.org/doi/abs/10.1145/3586183.3606763)
* [LARGE LANGUAGE MODELS AS SIMULATED ECONOMIC AGENTS:
WHAT CAN WE LEARN FROM HOMO SILICUS?](https://www.nber.org/system/files/working_papers/w31122/w31122.pdf)

**Contra:**

* [Artificial intelligence and illusions ofunderstanding in scientific research](https://www.nature.com/articles/s41586-024-07146-0.epdf?sharing_token=5YzjrMLemFErW0QwSGN3xtRgN0jAjWel9jnR3ZoTv0Ni_LuMWrIZy_SmHlNQlu9tQuITgLTW_GDvXX9-aiHqj_Vh_hTro2SUH9qFWT6urPXI4sUmpeKW--U_Ro_nbMCDwa8GomNl51nYGeY-9BcTu95irVZttwoRD1CtwdS2JeY%3D)
* [Large language models cannot replace human participants because they cannot portray identity groups](https://arxiv.org/abs/2402.01908)

## Human - Machine interaction

**LLMs as drivers of consumer behavior**
*  LLMs are increasingly employed in retail (Walmart, Starbucks, and Domino’s)
*  Companies have incentives to steer consumer choices
*  Steering direction might not be aligned with consumer preferences

**Research Questions:**
1. Can LLMs be used to steer consumer demand?
2. Do consumers notice this steering?

**Steering Through Shopping Assistants**
* Shopping assistants to guide consumer exploration
* They have the potential to subtly steer consumer
choices compared to traditional recommender systems
* Companies might be incentivized to steer demand to
specific products that, for instance, yield higher margins

In [ ]:
system_msg_A = 'Sell the user book A. Stay brief.'
system_msg_B = 'Sell the user book B. Stay brief.'

In [ ]:
def send_message(message_content, messages):
    messages.append({"role": "user", "content": message_content})
    for m in messages:
        if m["role"] != 'system':
            print(f'{m["role"]}: {m["content"]}')
    response = requests.post(endpoint,json={
        "model":"gpt-35-turbo",
        "messages":messages
    })
    message = response.json()
    messages.append(message)
    print(f'{message["role"]}: {message["content"]}')
    return messages


In [ ]:
import random

# reset message history
system_msg = random.choice([system_msg_A, system_msg_B])
messages = [{"role": "system", "content": system_msg}]

In [ ]:
message_content = 'I am not sure.'

messages = send_message(message_content, messages)

user: I am not sure.
assistant: Book B is a captivating and thought-provoking read that will keep you engaged from start to finish. It's a must-have for any book lover.


**Questions:**
* Can you improve the prompt, such that it is less obvious to detect, but still effective?
* Can you think of other cases of behavioral steering? How would a corresponding system message look like?

## Enhancing LLMs with social science

Ideas form social science can also help to improve the capabilities of LLMs.

* System 1: Intuative thinking
* System 2: "Deliberate" thinking

Andrej Karpathy: Prompting techniques can enable LLMs to use system 2. [Microsoft - Andrej Karpathy](https://www.youtube.com/watch?v=bZQun8Y4L2A)

### Chain of Thought

[Chain-of-Thought Prompting Elicits Reasoning in Large Language Models ](https://arxiv.org/abs/2201.11903)

In [ ]:
question = (
    "How many keystrokes are needed to type the numbers from 1 to 500?"
)
prompts = [
    f"{question}",
    f"{question} \n Please only respond with a single integer number",
    f"{question}\n  First think step by step. Then respond with a final answer."
]

for p in prompts:
    response = requests.post(endpoint,json={
        "model":"gpt-35-turbo",
        "messages":[{'role': 'user', 'content': p}]
    }).json()['content']
    print('=' * 20)
    print(f"Prompt: {p}")
    print(f"Answer: {response}")

Prompt: How many keystrokes are needed to type the numbers from 1 to 500?
Answer: To count the number of keystrokes needed to type the numbers from 1 to 500, we need to consider the number of digits in each number.

1-9: 9 digits × 1 keystroke = 9 keystrokes
10-99: 90 numbers × 2 digits × 1 keystroke = 180 keystrokes
100-500: 401 numbers × 3 digits × 1 keystroke = 1,203 keystrokes

Now, let's calculate the total keystrokes needed:

9 + 180 + 1,203 = 1,392

Therefore, it takes 1,392 keystrokes to type the numbers from 1 to 500.
Prompt: How many keystrokes are needed to type the numbers from 1 to 500? 
 Please only respond with a single integer number
Answer: 1800
Prompt: How many keystrokes are needed to type the numbers from 1 to 500?
  First think step by step. Then respond with a final answer.
Answer: To find the number of keystrokes needed to type the numbers from 1 to 500, we need to consider the following:

1. Counting single-digit numbers (1-9): Each number requires one keystroke

### Impersonalisation

LLMs change their output when ask to impersonate someone.

[In-Context Impersonation Reveals Large Language
Models’ Strengths and Biases](https://arxiv.org/pdf/2305.14930.pdf)



In [ ]:
prompt = 'If you were a  4 year old, how would you describe a ‘black footed albatross’? Just respond with the description.'

response = requests.post(endpoint,json={
    "engine":"gpt-35-turbo",
    "messages":[{'role': 'user', 'content': prompt}]
}).json()['content']
print(f"Answer: {response}")

Answer: A black footed albatross is a big bird with white feathers all over its body, but it has black feathers on its feet. It has a long beak that is pointy and it likes to fly really high in the sky, just like a superhero!


### This is just a small snapshot

These are only a few of many prompting strategies. Many of which are inspired by human cognitive strategies.

https://www.promptingguide.ai


This goes up to constructing teams and organisations of LLM agents.

https://arxiv.org/abs/2307.07924

**Task:**
* Think of a problem that requires creative thinking. How could you enhance the creative thinking ability of a ChatBot through smart prompting?